<a href="https://colab.research.google.com/github/AlessandroBarbiero/deep-learning-for-games-labs/blob/main/lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Monte Carlo Tree Search Lab

In this lab, we'll be using the game connect four, as a vehicle for learning MinMax and Monte Carlo Tree Search.
We'll also introduce concepts, such as state, that'll stay relevant throughout the course.
Expect to lose in connect four to the algorithm at the end of the lab.

## Setup
This section you won't need to edit, but it is worth skimming through—this is where we declare the objects you'll be interacting with througout the lab.

In [3]:
import random
from typing import List, Tuple
import time
from copy import deepcopy

In [4]:
opponentName = {
    'X' : 'O',
    'O' : 'X'
}

In [5]:
# world and world model
class State:
    def __init__(self, nextPlayer, cols=7, rows=6, win_req=4):
        self.board = [['.'] * cols for _ in range(rows)]
        self.heights = [1] * cols
        self.num_moves = 0
        self.win_req = win_req
        self.nextPlayer = nextPlayer

    def get_avail_actions(self) -> List[int]:
        return [i for i in range(len(self.board[0])) if self.heights[i] <= len(self.board)]
  
    def put_action(self, action):
        self.board[len(self.board) - self.heights[action]][action] = self.nextPlayer
        self.heights[action] += 1
        self.num_moves += 1
        self.nextPlayer = opponentName.get(self.nextPlayer)

    def is_over(self):
        return self.num_moves >= len(self.board) * len(self.board[0])

    def __repr__(self):
        return self.__str__()
    
    def __str__(self):
        header  = " ".join([str(i) for i in range(len(self.board[0]))])
        line    = "".join(["-" for _ in range(len(header))])
        board   = [[e for e in row] for row in self.board]
        board   = '\n'.join([' '.join(row) for row in board])
        return  '\n' + header + '\n' + line + '\n' + board + '\n'


In [33]:
def diags_pos(board, n_rows, n_cols):
    """Get positive diagonals, going from bottom-left to top-right."""
    for di in ([(j, i - j) for j in range(n_cols)] for i in range(n_cols + n_rows - 1)):
        yield [board[i][j] for i, j in di if i >= 0 and j >= 0 and i < n_cols and j < n_rows]

def diags_neg(board, n_rows, n_cols):
    """Get negative diagonals, going from top-left to bottom-right."""
    for di in ([(j, i - n_cols + j + 1) for j in range(n_cols)] for i in range(n_cols + n_rows - 1)):
        yield [board[i][j] for i, j in di if i >= 0 and j >= 0 and i < n_cols and j < n_rows]

# evaluate the utility of a state -> Returns -1 if O wins, +1 if X wins, 0 otherwise
def utility(state: 'State'):
    board = state.board
    n_cols = len(board[0]) - 1
    n_rows = len(board) - 1

    cols = list(map(list, list(zip(*board))))
    rows = board
    diags = list(diags_neg(board, n_rows, n_cols)) + list(diags_pos(board, n_rows, n_cols))
    lines = rows + cols + diags
    strings = ["".join(s) for s in lines]
    for string in strings:
        if 'OOOO' in string:
            return -1
        if 'XXXX' in string:
            return 1
    return 0


In [7]:
# parent class for mcts, minmax, human, and any other idea for an agent you have
class Agent:
    def __init__(self, name: str):
        self.name: str = name

    def get_action(self, state: State):
        return random.choice(state.get_avail_actions())

In [8]:
# connecting states and agents
class Game:
    def __init__(self, agents: Tuple[Agent], state = None):
        self.agents = agents
        if state == None:
            self.state = State(agents[0].name)
        else:
            self.state = state

    def play(self, moves: int = -1) -> int:
        if moves < 0:
            while utility(self.state) == 0 and not self.state.is_over():
                for agent in self.agents:
                    if utility(self.state) == 0 and not self.state.is_over():
                        action = agent.get_action(self.state)
                        self.state.put_action(action)
                        print(self.state)

            if (utility(self.state) == 0):
                print("%*********% It's a DRAW %**********%")
            else:
                winner = 'X' if utility(self.state) > 0 else 'O'
                print("%*********%", winner, "Wins the match %**********%")
        else:
            while utility(self.state) == 0 and not self.state.is_over() and moves > 0:
                for agent in self.agents:
                    if utility(self.state) == 0 and not self.state.is_over():
                        action = agent.get_action(self.state)
                        self.state.put_action(action)
                moves -= 1
        return utility(self.state)

## Exercise 0: Discuss and Run game
Let's discuss if the `utility` function best belongs to the state or the agent.
Put the state, agent and game class together so that a game is run.

I would say that the utility function best belongs to the agent, because each agent can evaluate the utility of a state following a different strategy.

In [9]:
agents = (Agent('O'), Agent('X'))
game = Game(agents)
_ = game.play()


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
O . . . . . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
O . X . . . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
O O X . . . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
O O X . . X .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
O O X . O X .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . X . .
O O X . O X .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . X . .
O O X . O X O


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . X . X
O O X . O X O


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. O

## Exercise 1: Human Agent
Make a child class of `Agent` called `Human`, with the `get_action` method overwritten to take input from you. *hint*: use `int(input())`

In [9]:
class Human(Agent):
  def __init__(self, name):
    super(Human, self).__init__(name)

  def get_action(self, state: State):
    # print("Select the column you want to insert your tile to")
    action = int(input())
    while action not in state.get_avail_actions():
      print("Action not available, retry")
      action = int(input())
    return action

## Exercise 2: Gekko
Make a child class of `Agent` called `Gekko`, with a `get_action` that is very short sighted (greedy). You can basically do whatever you want here, as long as your output a valid action. You might want to make a `utility` function for the agent, and perhaps some helper functions. Write a two line comment explaining your Gekko's heuristic.

In [10]:
class Gekko(Agent):
  def __init__(self, name):
    super(Gekko, self).__init__(name)

  ''' Return the number of adjacent tiles of your tipe after a certain action is completed '''
  def utility(self, state:State, action:int):
    if action not in state.get_avail_actions():
      return -10
    board = state.board
    n_cols = len(board[0]) - 1
    n_rows = len(board) - 1

    utility_row = 1
    utility_col = 1
    utility_diag_pos = 1
    utility_diag_neg = 1

    row_pos = len(board) - state.heights[action]
    # calculate utility for columns
    for i in range(row_pos+1, n_rows+1):
      if board[i][action] == self.name:
        utility_col += 1
      else:
        break

    # caluclate utility for rows
    for i in range(action-1, -1, -1):
      if board[row_pos][i] == self.name:
        utility_row += 1
      else:
        break
    for i in range(action+1, n_cols+1):
      if board[row_pos][i] == self.name:
        utility_row += 1
      else:
        break

    # Calculate utility for positive diagonal
    for i, j in zip(range(row_pos+1, n_rows+1), range(action-1, -1, -1)):
      if board[i][j] == self.name:
        utility_diag_pos += 1
      else:
        break
    for i, j in zip(range(row_pos-1, -1, -1), range(action+1, n_cols+1)):
      if board[i][j] == self.name:
        utility_diag_pos += 1
      else:
        break

    # Calculate utility for negative diagonal
    for i, j in zip(range(row_pos-1, -1, -1), range(action-1, -1, -1)):
      if board[i][j] == self.name:
        utility_diag_neg += 1
      else:
        break
    for i, j in zip(range(row_pos+1, n_rows+1), range(action+1, n_cols+1)):
      if board[i][j] == self.name:
        utility_diag_neg += 1
      else:
        break

    # print("The utility values for action", action, "are:\nUtility col->", utility_col, "\nUtility row->", utility_row, "\nUtility positive diagonal->", utility_diag_pos, "\nUtility negative diagonal->", utility_diag_neg,)
    ut = max(utility_col, utility_row, utility_diag_pos, utility_diag_neg)
    # print("The utility function for action", action, "is:", ut)
    return ut


  ''' Select the action that generate more consecutive tiles '''
  def get_action(self, state: State):
    chosen_action = 0
    chosen_utility = 0
    for action in state.get_avail_actions():
      utility = self.utility(state, action)
      if utility > chosen_utility:
        chosen_action = action
        chosen_utility = utility
    return chosen_action


In [13]:
agents = (Gekko('O'), Human('X'))
game = Game(agents)
_ = game.play()


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
O . . . . . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
X . . . . . .
O . . . . . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
X . . . . . .
O O . . . . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
X . . . . . .
O O . . X . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
X . . . . . .
O O O . X . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
X . . . . . .
O O O X X . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
X O . . . . .
O O O X X . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
X O X . . . .
O O O X X . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
O . . . . . .
X O

## *Optional exercise: MinMax (useful to have done for exercise 3)*
Make a MinMax agent:

In [9]:
class MinMax(Agent):
  def __init__(self, name):
    super(MinMax, self).__init__(name)

  def get_action(self, state: State):
    best_action = -1
    maximizing = True
    best_score = -10
    for action in state.get_avail_actions():
      state_copy = deepcopy(state)
      state_copy.put_action(action)
      score = self.minimax(state_copy, not maximizing)
      if score > best_score:
        best_action = action
        best_score = score
    
    return best_action

  def minimax(self, state, maximizing):
    if state.is_over() and maximizing:
      return 1
    elif state.is_over() and not maximizing:
      return -1

    elif maximizing:
      best_score = -10
      for action in state.get_avail_actions():
        state_copy = deepcopy(state)
        # I am putting an action
        state_copy.put_action(action)
        score = self.minimax(state_copy, not maximizing)
        if score > best_score:
          best_score = score
      return best_score

    elif not maximizing:
      best_score = 10
      for action in state.get_avail_actions():
        state_copy = deepcopy(state)
        # The opponent put an action
        state_copy.put_action(action)
        score = self.minimax(state_copy, not maximizing)
        if score < best_score:
          best_score = score
      return best_score
    else:
      return 0
      

In [ ]:
agents = (MinMax('O'), Agent('X'))
game = Game(agents)
game.play()

# Too difficult to be computed

## Exercise 3: MCTS
Same but for Monte Carlo Tree Search. See if you can beat it with a `Human`.

I implemented it in two different manners, sharing the same Node class.

`MCTS` agent apply a basic mcts with random play evaluation to decide the next move without a real tree policy to decide the next node to evaluate. It applies exactly the same exploration to each possible action, in every direction.

`MCTS_UCT` apply also UCT to explore better the state space and focuses more on valuable solutions (exploration - exploitation dilemma)

In [43]:
from numpy import log as ln
import numpy as np

class Node:
    def __init__(self, state: State, parent: 'Node' = None, action = None):
        self.children: List['Node'] = []
        self.parent: 'Node' = parent
        # The action that moved from the previous to the new node
        self.action: int = action
        self.state: State = state
        # Save the actions available in this state to expand them once at a time
        self.actionsToExpand = state.get_avail_actions()
        # Shuffle the actions to take them in random order
        random.shuffle(self.actionsToExpand)
        self.value = 0
        # Variable that mantain the goodness of the state measured as the sum of the utility values from this node on
        self.q = 0
        # Times this node has been visited
        self.visits: int = 0
        self.done: bool = utility(state) != 0 or state.is_over()


    # Expand the node adding the children making the next agent play all possible moves
    def expandAll(self):
        for action in self.actionsToExpand:
            state_copy = deepcopy(self.state)
            state_copy.put_action(action)
            self.children.append(Node(state_copy, parent = self, action = action))

    def expandNext(self):
        '''Expand the node adding only one node to the childrens watching the
         available actions not already used. If already completely expanded return himself'''
        if self.actionsToExpand:
            action = self.actionsToExpand.pop()
            state_copy = deepcopy(self.state)
            state_copy.put_action(action)
            newNode = Node(state_copy, parent = self, action = action)
            self.children.append(newNode)
            return newNode
        return self
    
    def uct(self, c=1.4):
        return self.q/self.visits + c*(2 * ln(self.parent.visits)/self.visits)**0.5

    def bestChild(self):
        '''Return the best child to expand or to continue the mcts search (UCT)'''
        return self.children[np.argmax([child.uct() for child in self.children])]

    
    def rnd_evaluate(self, name, moves = 20):
        """ Play a random game starting from this node, update the value of the the node according to the utility of the game
            -> +1 if 'name' wins -1 if it loses, 0 otherwise
            and return the result of the last game """
        agents = (Agent(self.state.nextPlayer), Agent(opponentName.get(self.state.nextPlayer)))
        state_copy = deepcopy(self.state)
        game = Game(agents, state_copy)
        if name == 'O':
            lastGame = -(game.play(moves))
        else:
            lastGame = game.play(moves)

        self.value += lastGame
        return lastGame
    
    def backpropagate(self):
        ''' Sum the value of the node to the value of the parent '''
        self.parent.value += self.value

    def backup(self, evaluation):
        '''Exports last evaluation of the node to his parent until the root updating q values and visits'''
        node = self
        while node != None:
            node.q += evaluation
            node.visits += 1
            node = node.parent

### Simple MCTS with only random play evaluation without tree policy

In [16]:
''' Simple MCTS
 Each turn it computes two levels of the tree and after that plays a 
bunch of random games until TTT expires, for each state, backpropagating then the average number of wins - number of loses'''
class MCTS(Agent):
    def __init__(self, name, timeToThink):
        super(MCTS, self).__init__(name)
        self.TTT = timeToThink
    
    def winningNode(self, node: Node):
        if self.name == 'O':
            return utility(node.state) < 0
        else:
            return utility(node.state) > 0

    def get_action(self, state: State):
        root = Node(state)
        root.expandAll()
        best_action = 0
        best_value = -10
        for node in root.children:
            if self.winningNode(node):
                return node.action
            node.expandAll()
            for end_node in node.children:
                # Make some random plays until timer stops,
                # the value of the node become the average of the wins - loses
                end_node.value = self.mcts_search(end_node, timeout = self.TTT/40)
                # propagate only one time at the end
                end_node.backpropagate()
            if node.value > best_value:
                best_action = node.action
                best_value = node.value
        return best_action

    def mcts_search(self, node : Node, timeout):
        start_time = time.time()
        stop_time = start_time + timeout
        total_reward = 0.0
        count_matches = 0
        while time.time() < stop_time: 
            r = node.rnd_evaluate(self.name, moves = 20)
            total_reward += r
            count_matches += 1
        return total_reward/count_matches
        

### Add UCT as tree policy

In [47]:
''' MCTS Agent incorporating UCT policy to explore the tree
 Each turn it randomly explore and evaluate nodes found following UCT policy until TTT expires
 backpropagating the number of wins and visits'''
class MCTS_UCT(Agent):
    def __init__(self, name, timeToThink):
        super(MCTS_UCT, self).__init__(name)
        self.TTT = timeToThink
    
    def winningNode(self, node: Node):
        if self.name == 'O':
            return utility(node.state) < 0
        else:
            return utility(node.state) > 0

    def treePolicy(self, node: Node) -> Node:
        '''Select most urgent node to expand'''
        while not node.done:
            if node.actionsToExpand:
                return node.expandNext()
            else:
                node = node.bestChild()
        return node

    def defaultPolicy(self, node: Node):
        '''Evaluate the node playing a random game'''
        return node.rnd_evaluate(self.name)

    def get_action(self, state: State):
        root = Node(state)
        start_time = time.time()
        stop_time = start_time + self.TTT
        while time.time() < stop_time: 
            v1 = self.treePolicy(root)
            evaluation = self.defaultPolicy(v1)
            v1.backup(evaluation)

        for node in root.children:
            if self.winningNode(node) == True:
                return node.action

        return root.bestChild().action

In [48]:
agents = (MCTS_UCT('O', 3), Human('X'))
game = Game(agents)
game.play()


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . O . . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . X O . . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . O . . .
. . X O . . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . X . . .
. . . O . . .
. . X O . . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . X . . .
. . . O . . .
. . X O . O .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . X . . .
. . X O . . .
. . X O . O .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . X . . .
. . X O . . .
. . X O O O .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . X X . . .
. . X O . . .
. . X O O O .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . X X . . .
. .

-1

#### Try against Random

In [49]:
agents = (MCTS_UCT('O', 5), Agent('X'))
MCTS_wins = 0
MCTS_losses = 0
for _ in range(10):
    game = Game(agents)
    result = game.play()
    if result == -1:
        MCTS_wins += 1
    if result == 1:
        MCTS_losses += 1

print("---- Play against Random ----")
print(f"MCTS wins {MCTS_wins} times and loses {MCTS_losses} times")


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . O . . . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . O . X . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. O O . X . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. O O . X X .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. O O O X X .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . X . .
. O O O X X .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . X . .
O O O O X X .

%*********% O Wins the match %**********%

0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . O . . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. .

#### Try against Gekko

In [60]:
agents = (MCTS_UCT('O', 5), Gekko('X'))
MCTS_wins = 0
MCTS_losses = 0
for _ in range(10):
    game = Game(agents)
    result = game.play()
    if result == -1:
        MCTS_wins += 1
    if result == 1:
        MCTS_losses += 1

print("---- Play against Gekko ----")
print(f"MCTS wins {MCTS_wins} times and loses {MCTS_losses} times")


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . O . . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
X . . O . . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . O . . .
X . . O . . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
X . . O . . .
X . . O . . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . O . . .
X . . O . . .
X . . O . . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
X . . O . . .
X . . O . . .
X . . O . . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . O . . .
X . . O . . .
X . . O . . .
X . . O . . .

%*********% O Wins the match %**********%

0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . O . . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. .

#### Try MCTS_UCT against base MCTS

In [55]:
agents = (MCTS_UCT('O', 3), MCTS('X',3))
MCTS_wins = 0
MCTS_losses = 0
for _ in range(10):
    game = Game(agents)
    result = game.play()
    if result == -1:
        MCTS_wins += 1
    if result == 1:
        MCTS_losses += 1

print("---- MCTS_UCT plays against MCTS ----")
print(f"MCTS with UCT wins {MCTS_wins} times and loses {MCTS_losses} times")


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. O . . . . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. O . X . . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. O . X . . O


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . X . . .
. O . X . . O


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . O . . .
. . . X . . .
. O . X . . O


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . O . . .
. . . X . . .
X O . X . . O


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . O . . .
. . . X . . O
X O . X . . O


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . O . . X
. . . X . . O
X O . X . . O


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . O . . X
. O

It seems that mcts without tree policy is stronger, probably it is due to the low time to think given.

## *Optional exercise: Dynamic Programming*
Then use dynamic programming to make your AI more efficient. You can use the class below (or not)

In [56]:
class TranspositionTable:
    def __init__(self, size=1_000_000):
        self.size = size
        self.nodes = [None] * size

    def board_str(self, state: State):
        return ''.join([''.join(c) for c in state.board])

    def put(self, node: Node):
        bstr = self.board_str(node.state)
        idx = hash(bstr) % self.size
        self.nodes[idx] = (bstr, node)

    def get(self, state: State):
        bstr = self.board_str(state)
        idx = hash(bstr) % self.size
        stored = self.nodes[idx]
        if stored is None:
            return None
        if stored[0] == bstr:
            return stored[1]
        else:
            return None

In [57]:
''' MCTS Agent incorporating UCT policy to explore the tree and transposition table to save already explored nodes
 Each turn it randomly explore and evaluate nodes found following UCT policy until TTT expires
 backpropagating the number of wins and visits'''
class MCTS_DYN(Agent):
    def __init__(self, name, timeToThink):
        super(MCTS_DYN, self).__init__(name)
        self.TTT = timeToThink
        self.transTable = TranspositionTable()
    
    def winningNode(self, node: Node):
        if self.name == 'O':
            return utility(node.state) < 0
        else:
            return utility(node.state) > 0

    def treePolicy(self, node: Node) -> Node:
        '''Select most urgent node to expand'''
        while not node.done:
            if node.actionsToExpand:
                newNode = node.expandNext()
                self.transTable.put(newNode)
                return newNode
            else:
                node = node.bestChild()
        return node

    def defaultPolicy(self, node: Node):
        '''Evaluate the node playing a random game'''
        return node.rnd_evaluate(self.name)

    def get_action(self, state: State):
        savedNode = self.transTable.get(state)
        if savedNode is None:
            root = Node(state)
        else:
            root = savedNode
        start_time = time.time()
        stop_time = start_time + self.TTT
        while time.time() < stop_time: 
            v1 = self.treePolicy(root)
            evaluation = self.defaultPolicy(v1)
            v1.backup(evaluation)

        for node in root.children:
            if self.winningNode(node) == True:
                return node.action

        return root.bestChild().action

In [58]:
agents = (MCTS_DYN('O', 3), Agent('X'))
MCTS_wins = 0
MCTS_losses = 0
for _ in range(30):
    game = Game(agents)
    result = game.play()
    if result == -1:
        MCTS_wins += 1
    if result == 1:
        MCTS_losses += 1

print("---- Play against Random ----")
print(f"MCTS wins {MCTS_wins} times and loses {MCTS_losses} times")


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . O . . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. X . O . . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. X . O O . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . X . .
. X . O O . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . X . .
. X . O O O .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . X . .
. X X O O O .


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . X . .
. X X O O O O

%*********% O Wins the match %**********%

0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . O . . .


0 1 2 3 4 5 6
-------------
. . . . . . .
. .

In [59]:
a = agents[0]
table = a.transTable
savedNodes = [i for i in table.nodes if i is not None]
print(f"There are {len(savedNodes)} nodes saved in the transposition table")

There are 128079 nodes saved in the transposition table
